# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
epoch = datetime.datetime(1960, 1, 1)
from pyspark.sql import SparkSession

from pyspark.sql.functions import udf, col
import pyspark.sql.types as T

import io
import os
import configparser
import boto3

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
# Read in the data here


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.


#### Immigration Data

In [4]:
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

#config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")

In [4]:
#write to parquet
df_spark.write.parquet("sas_data_2")

In [15]:
df_spark.printSchema()

root
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- fltno: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)



##### Weather Data

In [9]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(fname)
df.Country.unique()
df.dtypes

dt                                object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                              object
Country                           object
Latitude                          object
Longitude                         object
dtype: object

In [24]:
df_us = df.loc[(df['Country'] == "United States")]

df_us['dt'] = pd.to_datetime(df_us['dt'],format='%Y-%m-%d')

df_us['year'] = df_us['dt'].dt.year
df_us['month'] = df_us['dt'].dt.month
df_us['day'] = df_us['dt'].dt.day
df_us.dtypes

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

dt                               datetime64[ns]
AverageTemperature                      float64
AverageTemperatureUncertainty           float64
City                                     object
Country                                  object
Latitude                                 object
Longitude                                object
year                                      int64
month                                     int64
day                                       int64
dtype: object

In [25]:
df_us.loc[(df_us['City'] == "Miami") & (df_us['year'] == 2010)]

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year,month,day
4805290,2010-01-01,14.773,0.336,Miami,United States,26.52N,80.60W,2010,1,1
4805291,2010-02-01,15.479,0.139,Miami,United States,26.52N,80.60W,2010,2,1
4805292,2010-03-01,17.578,0.204,Miami,United States,26.52N,80.60W,2010,3,1
4805293,2010-04-01,22.778,0.283,Miami,United States,26.52N,80.60W,2010,4,1
4805294,2010-05-01,26.882,0.240,Miami,United States,26.52N,80.60W,2010,5,1
4805295,2010-06-01,29.336,0.223,Miami,United States,26.52N,80.60W,2010,6,1
4805296,2010-07-01,29.344,0.263,Miami,United States,26.52N,80.60W,2010,7,1
4805297,2010-08-01,29.211,0.286,Miami,United States,26.52N,80.60W,2010,8,1
4805298,2010-09-01,28.261,0.147,Miami,United States,26.52N,80.60W,2010,9,1
4805299,2010-10-01,24.575,0.176,Miami,United States,26.52N,80.60W,2010,10,1


##### Cities

In [2]:
fname = 'us-cities-demographics.csv'
df = pd.read_csv(fname,sep=';')
df.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [3]:
df.dtypes

City                       object
State                      object
Median Age                float64
Male Population           float64
Female Population         float64
Total Population            int64
Number of Veterans        float64
Foreign-born              float64
Average Household Size    float64
State Code                 object
Race                       object
Count                       int64
dtype: object

In [30]:
df.loc[(df_us['City'] == "New York")]

NameError: name 'df' is not defined

##### Airports

In [16]:
fname = 'airport-codes_csv.csv'
df = pd.read_csv(fname)
df.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [33]:
df.dtypes

ident            object
type             object
name             object
elevation_ft    float64
continent        object
iso_country      object
iso_region       object
municipality     object
gps_code         object
iata_code        object
local_code       object
coordinates      object
dtype: object

In [17]:
df_us = df.loc[(df['iso_country'] == "US")]
df_us_airports = df_us[["iso_region","name","municipality","local_code","type","coordinates"]]

In [39]:
df_us_airports.head(5)

,iso_region,name,municipality,local_code,type,coordinates
0,US-PA,Total Rf Heliport,Bensalem,00A,heliport,"-74.93360137939453, 40.07080078125"
1,US-KS,Aero B Ranch Airport,Leoti,00AA,small_airport,"-101.473911, 38.704022"
2,US-AK,Lowell Field,Anchor Point,00AK,small_airport,"-151.695999146, 59.94919968"
3,US-AL,Epps Airpark,Harvest,00AL,small_airport,"-86.77030181884766, 34.86479949951172"
4,US-AR,Newport Hospital & Clinic Heliport,Newport,NaN,closed,"-91.254898, 35.6087"


In [18]:
df_us_airports['state_code'] = df_us_airports['iso_region'].str[3:5]
df_us_airports.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,iso_region,name,municipality,local_code,type,coordinates,state_code
0,US-PA,Total Rf Heliport,Bensalem,00A,heliport,"-74.93360137939453, 40.07080078125",PA
1,US-KS,Aero B Ranch Airport,Leoti,00AA,small_airport,"-101.473911, 38.704022",KS
2,US-AK,Lowell Field,Anchor Point,00AK,small_airport,"-151.695999146, 59.94919968",AK
3,US-AL,Epps Airpark,Harvest,00AL,small_airport,"-86.77030181884766, 34.86479949951172",AL
4,US-AR,Newport Hospital & Clinic Heliport,Newport,NaN,closed,"-91.254898, 35.6087",AR



### Cleaning Steps
Document steps necessary to clean the data

#### Airports

- Re-arrange order of columns (remove iso_region) \n
- Delete duplicates

In [9]:
# Performing cleaning tasks here
fname = 'airport-codes_csv.csv'
df = pd.read_csv(fname)
df_us = df.loc[(df['iso_country'] == "US")]
df_us_airports = df_us[["iso_region","name","municipality","local_code","type","coordinates"]]
df_us_airports['state_code'] = df_us_airports['iso_region'].str[3:5]
df_us_airports["cityState"] = df_us_airports["state_code"]+df_us_airports["municipality"]
df_us_airports['cityState'] = df_us_airports['cityState'].str.lower()
df_us_airports.rename({'municipality': 'city'}, axis=1, inplace=True)
df_us_airports = df_us_airports[["cityState","state_code","name","city","local_code","type","coordinates"]]

df_us_airports.head(10)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

,cityState,state_code,name,city,local_code,type,coordinates
0,pabensalem,PA,Total Rf Heliport,Bensalem,00A,heliport,"-74.93360137939453, 40.07080078125"
1,ksleoti,KS,Aero B Ranch Airport,Leoti,00AA,small_airport,"-101.473911, 38.704022"
2,akanchor point,AK,Lowell Field,Anchor Point,00AK,small_airport,"-151.695999146, 59.94919968"
3,alharvest,AL,Epps Airpark,Harvest,00AL,small_airport,"-86.77030181884766, 34.86479949951172"
4,arnewport,AR,Newport Hospital & Clinic Heliport,Newport,NaN,closed,"-91.254898, 35.6087"
5,okalex,OK,Fulton Airport,Alex,00AS,small_airport,"-97.8180194, 34.9428028"
6,azcordes,AZ,Cordes Airport,Cordes,00AZ,small_airport,"-112.16500091552734, 34.305599212646484"
7,cabarstow,CA,Goldstone /Gts/ Airport,Barstow,00CA,small_airport,"-116.888000488, 35.350498199499995"
8,cabiggs,CA,Williams Ag Airport,Biggs,00CL,small_airport,"-121.763427, 39.427188"
9,capine valley,CA,Kitchen Creek Helibase Heliport,Pine Valley,00CN,heliport,"-116.4597417, 32.7273736"


#### Cities

Convert floats to int
Cut out into Cities / Races / Population
Remove duplicates

In [11]:
fname = 'us-cities-demographics.csv'
df_cities = pd.read_csv(fname,sep=';')
df_cities = df_cities.fillna(0)

df_cities['Male Population'] = df_cities['Male Population'].astype(int)
df_cities['Female Population'] = df_cities['Female Population'].astype(int)
df_cities['Number of Veterans'] = df_cities['Number of Veterans'].astype(int)
df_cities['Foreign-born'] = df_cities['Foreign-born'].astype(int)


df_cities['cityState'] = df_cities['State Code']+df_cities['City']
df_cities['cityState'] = df_cities['cityState'].str.lower()
df_races =  df_cities[['cityState','Race','Count']]

df_population = df_cities[['cityState','Median Age','Male Population','Female Population','Total Population','Number of Veterans','Foreign-born','Average Household Size']]
df_population = df_population.drop_duplicates()

df_cities = df_cities[['cityState','State Code','City','State']]
df_cities = df_cities.drop_duplicates()


In [4]:
df_cities.head(5)

,cityState,State Code,City,State
0,mdsilver spring,MD,Silver Spring,Maryland
1,maquincy,MA,Quincy,Massachusetts
2,alhoover,AL,Hoover,Alabama
3,carancho cucamonga,CA,Rancho Cucamonga,California
4,njnewark,NJ,Newark,New Jersey


In [5]:
df_test = df_cities.loc[df_cities['City'] == 'Abilene']
df_test

,cityState,State Code,City,State
245,txabilene,TX,Abilene,Texas


In [6]:
df_races.head(5)

,cityState,Race,Count
0,mdsilver spring,Hispanic or Latino,25924
1,maquincy,White,58723
2,alhoover,Asian,4759
3,carancho cucamonga,Black or African-American,24437
4,njnewark,White,76402


In [7]:
df_test = df_races.loc[df_races['cityState'] == 'txabilene']
df_test

,cityState,Race,Count
245,txabilene,American Indian and Alaska Native,1813
1403,txabilene,Hispanic or Latino,33222
1533,txabilene,White,95487
2727,txabilene,Asian,2929
2880,txabilene,Black or African-American,14449


In [8]:
df_population.head(5)

,cityState,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size
0,mdsilver spring,33.8,40601,41862,82463,1562,30908,2.60
1,maquincy,41.0,44129,49500,93629,4147,32935,2.39
2,alhoover,38.5,38040,46799,84839,4819,8229,2.58
3,carancho cucamonga,34.5,88127,87105,175232,5821,33878,3.18
4,njnewark,34.6,138040,143873,281913,5829,86253,2.73


In [9]:
df_test = df_population.loc[df_population['cityState'] == 'txabilene']
df_test

,cityState,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size
245,txabilene,31.3,65212,60664,125876,9367,8129,2.64


#### Immigration


In [2]:
fname = 'immigration_port.csv'
df_immigration_port = pd.read_csv(fname)
df_immigration_port['cityState'] = df_immigration_port['State_code']+df_immigration_port['City']
df_immigration_port['cityState'] = df_immigration_port['cityState'].str.lower()
df_immigration_port['City'] = df_immigration_port['City'].str.title()
df_immigration_port = df_immigration_port[['cityState','City','State_code','country','immigration_code']]


from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_may16_sub.sas7bdat')

immigration_table = df_spark.select(["i94port","i94yr","i94mon","arrdate","depdate","i94mode","i94addr","biryear","gender","occup","airline","fltno","entdepa","entdepd","entdepu","matflag","visatype"])

immigration_table = immigration_table.withColumnRenamed("i94port","entry_port") \
                        .withColumnRenamed("i94yr","year") \
                        .withColumnRenamed("i94mon","month") \
                        .withColumnRenamed("i94mode", "mode") \
                        .withColumnRenamed("biryear", "birthyear") \
                        .withColumnRenamed("occup", "occupation") \
                        .withColumnRenamed("fltno", "flightno") \
                        .withColumnRenamed("i94addr", "addr")

def convert_sas(sastime):
    try:
        return (pd.to_timedelta(sastime,unit='D')+ epoch)
    except:
        return None

convert_date = udf(lambda x: convert_sas(x),T.DateType())

immigration_table = immigration_table.withColumn('arrdate',convert_date(immigration_table.arrdate))\
                                    .withColumn('depdate',convert_date(immigration_table.depdate))

immigration_table = immigration_table.withColumn('year',col('year').cast('integer'))\
                .withColumn('month',col('month').cast('integer'))\
                .withColumn('birthyear',col('birthyear').cast('integer'))

Py4JJavaError: An error occurred while calling o45.load.
: java.lang.ClassNotFoundException: Failed to find data source: com.github.saurfang.sas.spark. Please find packages at http://spark.apache.org/third-party-projects.html
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:657)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:194)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:178)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.

In [3]:
df_immigration_port.head(5)

,cityState,City,State_code,country,immigration_code
0,akalcan,Alcan,AK,US,ALC
1,akanchorage,Anchorage,AK,US,ANC
2,akbaker aaf - baker island,Baker Aaf - Baker Island,AK,US,BAR
3,akdaltons cache,Daltons Cache,AK,US,DAC
4,akdew station pt lay dew,Dew Station Pt Lay Dew,AK,US,PIZ


In [4]:
immigration_table.printSchema()

NameError: name 'immigration_table' is not defined

In [80]:
immigration_table_pd = immigration_table.limit(10).toPandas()
immigration_table_pd

,entry_port,year,month,arrdate,depdate,mode,i94addr,birthyear,gender,occupation,airline,flightno,entdepa,entdepd,entdepu,matflag,visatype
0,XXX,2016,5,2016-05-31,None,NaN,None,1989,None,None,None,None,T,None,U,None,F1
1,XXX,2016,5,2016-05-24,None,NaN,None,1989,None,None,None,None,T,None,U,None,E2
2,XXX,2016,5,2016-05-04,None,NaN,None,1938,None,None,None,None,T,None,U,None,B2
3,XXX,2016,5,2016-05-27,None,NaN,None,1987,None,None,None,None,T,None,U,None,F1
4,CHI,2016,5,2016-05-03,2015-07-01,1.0,IL,1987,F,None,EK,00235,T,O,None,M,F1
5,XXX,2016,5,2016-05-31,None,0.0,None,1968,None,None,None,None,None,None,None,None,M1
6,BOS,2016,5,2016-05-01,2016-05-13,1.0,NY,1963,F,None,BA,215,T,O,None,M,B2
7,WAS,2016,5,2016-05-01,2016-05-10,1.0,NY,1950,None,None,OS,00093,O,O,None,M,B1
8,NYC,2016,5,2016-05-01,2016-05-30,1.0,NY,1948,None,None,AZ,00608,O,O,None,M,B2
9,MIA,2016,5,2016-05-01,2016-05-22,1.0,FL,1984,F,None,AZ,630,T,O,None,M,B2


#### Weather


In [2]:
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(fname)

df_us = df.loc[(df['Country'] == "United States")]

df_us['dt'] = pd.to_datetime(df_us['dt'],format='%Y-%m-%d')

df_us['year'] = df_us['dt'].dt.year
df_us['month'] = df_us['dt'].dt.month

df_us = df_us.loc[(df_us['year'] > 1999)]

df_weather = df_us[['City','AverageTemperature','year','month']]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

In [20]:
df_weather.head(10)

,City,AverageTemperature,year,month
49715,Abilene,8.039,2000,1
49716,Abilene,11.908,2000,2
49717,Abilene,14.423,2000,3
49718,Abilene,18.274,2000,4
49719,Abilene,25.358,2000,5
49720,Abilene,25.264,2000,6
49721,Abilene,29.421,2000,7
49722,Abilene,29.733,2000,8
49723,Abilene,25.446,2000,9
49724,Abilene,18.477,2000,10


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
config = configparser.ConfigParser()
config.read('dl.cfg')
aws_access_key_id=config['AWS']['AWS_ACCESS_KEY_ID']
aws_secret_access_key=config['AWS']['AWS_SECRET_ACCESS_KEY']
aws_session_token=config['AWS']['AWS_SESSION_TOKEN']
s3_bucket = "udacityexercisealeaume"

#HELP / SOURCE: https://towardsdatascience.com/reading-and-writing-files-from-to-amazon-s3-with-pandas-ccaf90bfe86c

s3_client = boto3.client(
    "s3",
    aws_access_key_id=os.environ['AWS_ACCESS_KEY_ID'],
    aws_secret_access_key=os.environ['AWS_SECRET_ACCESS_KEY'],
    aws_session_token=os.environ['AWS_SESSION_TOKEN'],
)
#Weather
with io.StringIO() as csv_buffer:
    df_weather.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=s3_bucket, Key="P6/output/weather/weather.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response for Weather file. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response for Weather file. Status - {status}")

#Airport
#df_us_airports.to_csv(output_s3_path +"output/airports/airports.csv")
with io.StringIO() as csv_buffer:
    df_us_airports.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=s3_bucket, Key="P6/output/airports/airports.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response for Airport file. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response for Airport file. Status - {status}")


#Population
#Cities
with io.StringIO() as csv_buffer:
    df_cities.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=s3_bucket, Key="P6/output/demographics/cities.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response for Cities file. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response for Cities file. Status - {status}")

#Races
with io.StringIO() as csv_buffer:
    df_races.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=s3_bucket, Key="P6/output/demographics/races.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response for Races file. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response for Races file. Status - {status}")
        
#Population
with io.StringIO() as csv_buffer:
    df_population.to_csv(csv_buffer, index=False)

    response = s3_client.put_object(
        Bucket=s3_bucket, Key="P6/output/demographics/population.csv", Body=csv_buffer.getvalue()
    )

    status = response.get("ResponseMetadata", {}).get("HTTPStatusCode")

    if status == 200:
        print(f"Successful S3 put_object response for Pupulation file. Status - {status}")
    else:
        print(f"Unsuccessful S3 put_object response for Population file. Status - {status}")
        


In [5]:
#Immigration (spark DF)
os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']
immigration_table.write.mode('overwrite').parquet("s3a://" + s3_bucket+"P6/outputoutput/immigration")
#immigration_table.write.parquet("s3a://" + s3_bucket+"P6/outputoutput/immigration")


NameError: name 'config' is not defined

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.